In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as torch_data
from torchsummary import summary
from torchvision import models
from pycocotools.coco import COCO
import torchvision.transforms as transforms
import json
import skimage.io as io
import matplotlib.pyplot as plt
import numpy as np

import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger

import os, sys
sys.path.append(os.path.abspath("../"))

%load_ext autoreload
%autoreload 2


from models.base_line.model import BaseLine
from models.autoencoders.components import ConvBlock, Bridge
from models.GANs.model import ConstrastDCGAN

/usr/local/anaconda3/lib/python3.8/site-packages/pytorch_lightning/metrics/__init__.py:43: LightningDeprecationWarning: `pytorch_lightning.metrics.*` module has been renamed to `torchmetrics.*` and split off to its own package (https://github.com/PyTorchLightning/metrics) since v1.3 and will be removed in v1.5
  rank_zero_deprecation(


In [2]:
class ToyDataset(torch_data.Dataset):
    def __init__(self, len=64):
        super().__init__()
        self.len = len
    
    def __getitem__(self, idx):
        return torch.ones(3, 128, 128), torch.ones(3, 128, 128)*2, torch.zeros(1)

    def __len__(self):
        return self.len

In [3]:
gen_train_dl    = torch_data.DataLoader(torch.ones(8, 3, 128, 128), batch_size=4)
disc_train_dl   = torch_data.DataLoader(torch.ones(8, 3, 64, 64), batch_size=4)
ae_train_dl     = torch_data.DataLoader(ToyDataset(8), batch_size=4)

val_dl          = torch_data.DataLoader(ToyDataset(8), batch_size=4) 
test_dl         = torch_data.DataLoader(ToyDataset(8), batch_size=4)

In [4]:
z_size = 2048

In [5]:
net = BaseLine(
            ae_output_size=(128, 128),
            ae_bridge_out_dims=[512],
            gan_noise_dim=512,
            ae_train_dl=ae_train_dl,
            disc_train_dl=disc_train_dl,
            val_dl=val_dl,
            test_dl=test_dl,
            ae_lr=1e-4, gan_lr=1e-4
        )

In [6]:
net.forward(*net.example_input_array).shape

torch.Size([3, 3, 64, 64])

In [7]:
real_batch=next(iter(disc_train_dl))
real_batch.shape

torch.Size([4, 3, 64, 64])

In [8]:
fake_batch=net._ae_forward(*next(iter(ae_train_dl)))["union"].flatten(1)
fake_batch.shape

torch.Size([4, 2048])

In [9]:
net.gan._disc_step(
    real=real_batch, 
    fake=fake_batch
)

tensor(1.5521, grad_fn=<AddBackward0>)

In [10]:
net.gan._gen_step(fake_batch)

tensor(0.1431, grad_fn=<BinaryCrossEntropyBackward>)

In [11]:
trainer = pl.Trainer(reload_dataloaders_every_epoch=True, max_epochs=2)
trainer.fit(net)

,  4.59s/it, loss=5.6, v_num=1]
Epoch 1:   0%|          | 0/4 [00:00<?, ?it/s, loss=5.6, v_num=1][tensor([[[[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]],

         [[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]],

         [[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]]],


        [[[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
         

OSError: [Errno 28] No space left on device

In [ ]:
trainer.test(net)

logger = TensorBoardLogger("tb_logs/", name="my_model", log_graph=True)
logger.log_graph(net, net.example_input_array)
%load_ext tensorboard
%tensorboard --logdir tb_logs

In [82]:
torch.Tensor(4)

tensor([ 0.0000e+00, -2.5244e-29,  0.0000e+00, -2.5244e-29])

In [76]:
torch.cat([torch.Tensor([0.1])]*2, 0).mean()

tensor(0.1000)